In [8]:
%%writefile train_veg_v1.py
### Version 2:  with automl
### Version 1:  Migrate EY code with minimal refractorying
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import time
from calendar import month_abbr
import os

### For accessing blob storage using our customised functions
import sys
# sys.path.append('..')
# sys.path.append('..util/space_blob')
from util.space_blob.space_blob.user_blob import create_blob_stream
from util.space_blob.space_blob.user_blob import write_blob_client
from util.space_blob.space_blob.user_blob import get_blobnames
from fsspec.implementations.http import HTTPFileSystem


### ML packages
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import GridSearchCV
import pickle as pkl
# import glob

import joblib
import argparse
from azureml.core import Run
run = Run.get_context()

# bb = get_blobnames('')

month_flt = 2
### change this to glob search later.. manual for now for testing purpose
## file = glob.glob('*pv_*.csv')
fn_list = ['HC_pv_100px_monthly_train.csv', 
           'Mesa_pv_100px_monthly_train.csv',
]

df_concat = []
for file in fn_list:
    
    fn_url_f = create_blob_stream(file, t_expire=8)
    df_tmp = pd.read_csv(fn_url_f)
    df_concat.append(df_tmp)
df_ori = pd.concat(df_concat)

df_use = df_ori.copy()  # working on a copy
# df_use.info() 

### clean up data for missing values
df_withnan = df_use[df_use.isna().any(axis=1)]
# display(df_withnan)

### I am going to drop NAN rows as they are only 4% of the data
df_use.dropna(inplace=True)

### Drop unused columns
dropped_names = ['Unnamed: 0', 'lat_4326', 'lon_4326']   ### no such columns for v2.csv
df_use.drop(dropped_names, axis=1, inplace=True)

### Choose feature columns
col_names = list(df_use.columns)
# print(col_names)

### Select ground truth column(s)
gr_truth_cols = ['actual_2m', 'actual_4m', 'actual_6m']
df_features = df_use.drop(gr_truth_cols, axis=1)
df_truths = df_use[gr_truth_cols]

### Drop the Year column as it cannot be a feature in forecasting models
X = df_features.iloc[:, 1:].values

if month_flt == 2:
    y_m = df_truths.iloc[:, 0].values
elif month_flt == 4:
    y_m = df_truths.iloc[:, 1].values
elif month_flt == 6:
    y_m = df_truths.iloc[:, 2].values
else:
    print('!!!Warning!!!!: Current model can only predict 2, 4, 6 months')
        
# print(X.shape, y_m.shape)
# print(X[:5, 0])
# print(X[:1, :])

### Encode categorical columns
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(sparse=False), [0])], remainder='passthrough')
X = np.array(ct.fit_transform(X))


### split the data into training and hand-out (testing):
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_m, test_size=0.2, random_state=123)

xgb1 = XGBRegressor()

parameters = {'nthread':[12], # max number of thread that can be used during training
              'objective':['reg:squarederror'],
              'learning_rate': [0.012], #so called `eta` value
              'max_depth': [12],    # default 6, higher = more complex, cause overfit
              'min_child_weight': [4],  # default 1, higher = prevent overfit
              'subsample': [0.8],   # default 1, smaller = prevent overfit
              'colsample_bytree': [1],  # default 1
              'gamma': [0],   # default 0, larger = more conservative
              'lambda': [1],    # default 1, as Ridge, larger=more conservative
              'alpha': [0],     # default 0, as Lasso, larger=more conservetive
              'n_estimators': [900]}


xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 3,
                        n_jobs = -1,
                        verbose=2)

xgb_grid.fit(X_train, y_train)
#  using best parameters from GridSearch
model_best_xgb =  xgb_grid.best_estimator_

path_parent = os.getcwd()
fo = os.path.join(path_parent, 'model_output')
if not os.path.exists(fo):
    os.makedirs(fo)
    
### save 
model_best_xgb.save_model(f'gen_best_xgb_{month_flt}mon_pv.json')

filename = f'{fo}+/gen_best_xgb_{month_flt}mon_pv_model.pkl'
# to save the model
# pkl.dump(model_best_xgb, open(filename, 'wb'))

### write into a pkl file
# joblib.dump(model_best_xgb, f'gen_best_xgb_{month_flt}mon_pv_model.pkl')
# joblib.dump(model_best_xgb, 'gen_best_xgb_2mon_pv_model.pkl')



if not os.path.exists('outputs'):
    os.makedirs('outputs')

model_name = f'gen_best_xgb_{month_flt}mon_pv_model.pkl'
filename = "outputs/" + model_name

joblib.dump(value=model_best_xgb, filename=filename)
run.upload_file(name=model_name, path_or_stream=filename)

# print('--- Model Performance Summary ---')

y_predict_train = model_best_xgb.predict(X_train)
y_predict_test = model_best_xgb.predict(X_test)

mse_train = mean_squared_error(y_train, y_predict_train)
# print("RMSE_train: %.2f" % (mse_train**(1/2.0)))
mse = mean_squared_error(y_test, y_predict_test)
# print("MSE_test: %.2f" % mse)
# print("RMSE_test: %.2f" % (mse**(1/2.0)))

score = model_best_xgb.score(X_train, y_train)  
# print('Training Score: ', np.round(score, 2))



from sklearn.metrics import r2_score
r2 = r2_score(y_test, y_predict_test)
print("R2 Score: %.2f" % r2)

run.log("MSE_test", mse)
run.log("R2_test Score", r2)



data_sns = pd.DataFrame({'y_test': y_test,
                         'ypred': y_predict_test,
                        })


# plt.rcParams['figure.figsize'] = [4, 4]
# x_ax = range(len(y_test))
# y_ax = x_ax
# # sns.scatterplot(x='y_test', y='ypred', data=data_sns, marker='x', alpha=1)
# sns.kdeplot(x='y_test', y='ypred', data=data_sns, cmap="Blues", shade=True)
# _ = plt.plot(x_ax, y_ax, color='grey', linestyle='--')
# _ = plt.title("XGBoost (Testing data: Green Veg) ")
# _ = plt.ylim(0, 1)
# _ = plt.xlim(0, 1)
# _ = plt.ylabel(f'Predicted EVI in {month_flt}mon (dex)')
# _ = plt.xlabel(f'Actual EVI in {month_flt}mon (dex)')
# _ = plt.show()

# plt.rcParams['figure.figsize'] = [6, 8]
# numx = X_train.shape[1]

# f_names = [None]*(X_train.shape[1])
# f_names[0:12]  = month_abbr[1:]
# f_names[12:] = df_features.columns.values[2:]

# feat_importances = pd.Series(model_best_xgb.feature_importances_, index=f_names)
# # feat_importances.nlargest(X_train.shape[1]).plot(kind='barh')
# feat_importances.nlargest(15).plot(kind='barh')

# _ = plt.xlabel('Relative importance')
# _ = plt.title('XGBoost Top15 Features')
# plt.show()

# joblib.dump(model_best_xgb, f'gen_best_xgb_{month_flt}mon_pv_model.pkl')

# g.figure.savefig(f'{fo}+/feature_int_corr.png', dpi=300, bbox_inches='tight')


run.complete()

Overwriting train_veg_v1.py
